In [1]:
import re
import time
import random
import numpy as np
import pandas as pd

from tqdm import tqdm
from musixmatch import Musixmatch

In [2]:
# API Key for Musixmatch
musixmatch = Musixmatch('c20977e27269ddaffa9bf5a0599e8246')

#Load MSD Sample DataFrame
df = pd.read_csv('data/MSD_sample.csv')
df.head()

,num_songs,artist_familiarity,artist_hotttnesss,artist_id,artist_mbid,artist_playmeid,artist_7digitalid,artist_latitude,artist_longitude,artist_location,...,sections_confidence,beats_start,beats_confidence,bars_start,bars_confidence,tatums_start,tatums_confidence,artist_mbtags,artist_mbtags_count,year
0,1,0.557460,0.386152,AREJXK41187B9A4ACC,c43bb0d6-94d7-410f-80fb-e5a243b18d23,2676,16971,46.71067,1.71819,France,...,[1. 1. 0.508 0.334 0.113 0.593],[4.2570000e-02 5.2458000e-01 1.0065900e+00 1.4...,[0.886 0.725 0.748 0.721 0.784 0.417 0.421 0.5...,[ 0.52458 2.4444 4.3658 6.30006 8.2...,[0.179 0.373 0.127 0.015 0.012 0.119 0.083 0.0...,[4.2570000e-02 2.8357000e-01 5.2458000e-01 7.6...,[0. 0. 0. 0. 0. 0. 0. 0. ...,[],[],2008
1,1,0.626958,0.434860,AR2XRFQ1187FB417FE,a69cd724-2f57-4ed0-bfed-ba20401eb84c,5772,92108,NaN,NaN,NaN,...,[1. 0.502 0.721 0.552 0.575 0.704 0.332 0.1...,[ 0.59161 1.33555 2.07949 2.82093 3.5...,[0.773 0.247 0.402 0.115 0.092 0.436 0.524 0.4...,[ 8.69527 12.01758 14.81363 20.09003 22.6...,[0.716 0.306 0.015 0.572 0.529 0.353 0.206 0.9...,[ 0.59161 0.84042 1.08799 1.33555 1.5...,[0.365 0.321 0.29 0.264 0.238 0.217 0.208 0.1...,[],[],2004
2,1,0.425724,0.000000,ARODOO01187FB44F4A,60bd8a1c-c093-4849-8f28-08101ca059b1,20540,1701,NaN,NaN,NaN,...,[1. 0.163 0.07 0.06 ],[ 1.95344 3.0622 4.16548 5.26467 6.35701 ...,[0.506 0.467 0.45 0.682 0.341 0.406 0.383 0.5...,[ 1.95344 6.35701 10.70579 15.07913 19.48322 ...,[0.014 0.025 0.049 0.029 0.11 0.484 0.357 0.0...,[ 1.11537 1.39701 1.67864 1.95344 2.23097 ...,[0.307 0.305 0.291 0.284 0.281 0.28 0.276 0.2...,[],[],0
3,1,0.611495,0.334520,ARJGW911187FB586CA,44b5b950-2ae2-403a-8c67-82d8fc72033d,-1,92184,NaN,NaN,NaN,...,[1. 0.744 0.36 0.628 0.147 0.093],[ 0.39309 1.17222 1.95902 2.75109 3.5...,[0.862 0.509 0.011 0.39 0.479 0.422 0.459 0.4...,[ 0.39309 2.75109 5.13649 7.52978 9.9...,[0.095 0.147 0.157 0.191 0.115 0.222 0.275 0.0...,[1.1929000e-01 3.9309000e-01 6.5603000e-01 9.1...,[0.731 0.628 0.526 0.359 0.287 0.246 0.21 0.1...,[],[],2007
4,1,0.367255,0.311616,AR9HQ6Y1187FB3C2CB,0e6524bd-6641-46a6-bce5-96f06c19aa46,-1,278655,NaN,NaN,NaN,...,[1. 0.171 0.658 0.609 0.551 0.591 0.525 0.6...,[ 0.49266 0.99301 1.49336 1.99371 2.4...,[0.382 0.374 0.464 0.343 0.553 0.49 0.63 0.5...,[ 0.99301 2.99441 4.96889 6.9402 8.9...,[0.32 0.081 0.567 0.328 0.112 0.004 0.09 0.1...,[ 0.24248 0.49266 0.74283 0.99301 1.2...,[0. 0. 0. 0. 0. 0. 0. 0. ...,[],[],0


In [3]:
musixmatch = Musixmatch('c20977e27269ddaffa9bf5a0599e8246')

list_of_songs = list(zip(df.title, df.artist_name))
cnt = 0

title = []
artist_name = []
track_id = []
album_id = []
instrumental = []
explicit = []
genre = []
lyrics = []
in_mxm = []

for title_, artist_name_ in tqdm(list_of_songs):
#     idx = random.randrange(0,9999)
#     print(idx)

    cnt+=1
#         musixmatch = Musixmatch('c20977e27269ddaffa9bf5a0599e8246')
#         break
    
    q_track = title_ #df['title'][idx]
    q_artist = artist_name_ #df['artist_name'][idx]
    title.append(title_)
    artist_name.append(artist_name_)
#     print(q_track)
#     print(q_artist)


    
    # mxm request based on artist and title.
    json_track_id = musixmatch.track_search(q_track=q_track, q_artist=q_artist, page_size=10, page=1, s_track_rating='desc')
    
    # Setting flags for positive request, available and if the song has lyrics.
    positive_request = json_track_id['message']['header']['status_code'] == 200
    is_available = json_track_id['message']['header']['available'] >= 1
    in_mxm.append(is_available)
    if is_available:
        has_lyrics = json_track_id['message']['body']['track_list'][0]['track']['has_lyrics'] == 1
    else:
        has_lyrics = False
#         lyrics.append(np.nan)
    
    
    # Checking if all the flags are True
    if positive_request & has_lyrics & is_available:
    
        # Save the variables of interest
        track_id_ = json_track_id['message']['body']['track_list'][0]['track']['track_id']
        track_id.append(track_id_)
        album_id.append(json_track_id['message']['body']['track_list'][0]['track']['album_id'])
        explicit.append(json_track_id['message']['body']['track_list'][0]['track']['explicit'])
        instrumental.append(json_track_id['message']['body']['track_list'][0]['track']['instrumental'])


        # Check if music genre is available
        if len(json_track_id['message']['body']['track_list'][0]['track']['primary_genres']['music_genre_list']) >= 1:
            genre.append(json_track_id['message']['body']['track_list'][0]['track']['primary_genres']['music_genre_list'][0]['music_genre']['music_genre_name'])
        else:
            genre.append(np.nan)
    
        # Request lyrics based on track_id
        json_lyrics = musixmatch.track_lyrics_get(track_id_)
        if json_lyrics['message']['header']['status_code'] == 200:
            lyrics_ = json_lyrics['message']['body']['lyrics']['lyrics_body']
            lyrics_ = lyrics_.replace("\n", " ")
            lyrics_ = lyrics_.replace("******* This Lyrics is NOT for Commercial use *******", "")
            lyrics_ = lyrics_.split("...")[0]
            lyrics.append(lyrics_)
#             print(lyrics_)
        else:
            pass
#             print('Request not processed')
    
    else:
        track_id.append(np.nan)
        album_id.append(np.nan)
        instrumental.append(np.nan)
        explicit.append(np.nan)
        genre.append(np.nan)
        lyrics.append(np.nan)
#         print('Positive Request: {}'.format(positive_request))
#         print('Is Available: {}'.format(is_available))
#         print('Has Lyrics: {}'.format(has_lyrics))
    if cnt%800 == 0:
        break




  8%|▊         | 799/10000 [02:42<31:09,  4.92it/s]  


In [4]:
len(title), len(artist_name), len(in_mxm), len(track_id), len(album_id), len(instrumental), len(explicit), len(genre), len(lyrics)

(800, 800, 800, 800, 800, 800, 800, 800, 800)

In [6]:
df_lyrics = pd.DataFrame({'title':title, 'artist_name':artist_name, 'in_mxm':in_mxm, 'track_id':track_id, 'album_id':album_id, 'instrumental':instrumental, 'explicit':explicit, 
              'genre':genre, 'lyrics':lyrics})

In [ ]:
musixmatch.track_search(q_track='Bennie', q_artist='Koffi ', page_size=10, page=1, s_track_rating='desc')